# Create SageMaker Endpoint from SageMaker Model Registry
아래 노트북에서 Model Registry 에 등록된 모델을 SageMaker Endpoint 로 배포하는 노트북 입니다.
- [01-ds-sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb](aws-ai-ml-workshop-kr/sagemaker/sm-pipeline/abalone_build_train_deploy/01-ds-sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

---

## 1. Model Package Group Name
- 이전 노트북에서 등록된 이름 입니다.
    - 필요시 이름을 수정 하세요.

In [ ]:
model_package_group_name = f"About-AbaloneModelPackageGroupName"

## 2. 모델 패키지 업데이트
- 등록된 Model 의 ModelApprovalStatus 상태가 Approved 가 아니어서 Approved 상태로 변경 합니다.

In [ ]:
import boto3

def update_model_approval_status(model_package_arn):
    try:
        sm_client = boto3.client('sagemaker')
        
        # 모델 패키지 승인 상태 업데이트 - 올바른 파라미터 이름 사용
        response = sm_client.update_model_package(
            ModelPackageArn=model_package_arn,  # ModelPackageName이 아닌 ModelPackageArn 사용
            ModelApprovalStatus="Approved",
            ApprovalDescription="Model approved for deployment"
        )
        
        print(f"모델 패키지 {model_package_arn}가 성공적으로 승인되었습니다.")
        return response
        
    except Exception as e:
        print(f"모델 승인 상태 업데이트 중 오류 발생: {e}")
        raise

# 사용 예시
try:
    # Model Registry에서 최신 모델 패키지 가져오기
    sm_client = boto3.client('sagemaker')
    model_packages = sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1
    )
    
    if not model_packages['ModelPackageSummaryList']:
        raise Exception("모델 패키지를 찾을 수 없습니다.")
    
    # 최신 모델 패키지 정보 가져오기
    model_package = model_packages['ModelPackageSummaryList'][0]
    model_package_arn = model_package['ModelPackageArn']
    
    print(f"현재 모델 승인 상태: {model_package['ModelApprovalStatus']}")
    
    # 모델 승인 상태 업데이트
    if model_package['ModelApprovalStatus'] != 'Approved':
        update_model_approval_status(model_package_arn)
        print("모델 승인이 완료되었습니다. 이제 엔드포인트를 생성할 수 있습니다.")
    else:
        print("모델이 이미 승인된 상태입니다.")
    
except Exception as e:
    print(f"오류 발생: {e}")
    raise

## 3. 모델 엔드포인트 배포

In [ ]:
import boto3
import sagemaker

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
role

In [ ]:
from sagemaker import ModelPackage
import time

def deploy_model_package(model_package_arn, endpoint_name, role):
    try:
        # 모델 패키지로부터 모델 생성
        model = ModelPackage(
            model_package_arn=model_package_arn,
            role=role,  # SageMaker 실행 역할 ARN
            sagemaker_session=sagemaker_session
        )
        
        # 엔드포인트 배포
        predictor = model.deploy(
            initial_instance_count=1,
            instance_type="ml.m5.xlarge",
            endpoint_name=endpoint_name
        )
        
        return predictor
    except Exception as e:
        print(f"엔드포인트 생성 중 오류 발생: {e}")
        raise

### 엔드 포인트 배포
- 아래 약 5분 걸립니다.

In [ ]:
%%time
import boto3 

try:
    # Model Registry에서 최신 승인된 모델 패키지 가져오기
    sm_client = boto3.client('sagemaker')
    model_packages = sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        # ModelApprovalStatus='Approved',
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1
    )
    print("model_packages: \n", model_packages)
    
    if not model_packages['ModelPackageSummaryList']:
        raise Exception("승인된 모델 패키지를 찾을 수 없습니다.")
        
    model_package_arn = model_packages['ModelPackageSummaryList'][0]['ModelPackageArn']
    endpoint_name = f"abalone-endpoint-{int(time.time())}"
    
    # 엔드포인트 생성
    endpoint = deploy_model_package(
        model_package_arn=model_package_arn,
        endpoint_name=endpoint_name,
        role=role
    )
    
    print(f"생성된 엔드포인트 이름: {endpoint}")
    
except Exception as e:
    print(f"오류 발생: {e}")
    raise

## 4. 엔드포인트에 추론

In [ ]:
import boto3
import numpy as np
import json

def predict_abalone(endpoint_name, input_data):
    """
    SageMaker 엔드포인트를 사용하여 전복 데이터 추론
    
    Parameters:
    endpoint_name (str): SageMaker 엔드포인트 이름
    input_data (list): [Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight]
    
    Returns:
    predictions: 추론 결과
    """
    try:
        # 입력 데이터에 성별(sex) 원-핫 인코딩 추가
        # 예시로 'M'(수컷)으로 가정 [M, F, I] -> [1, 0, 0]
        one_hot_encoded = [1, 0, 0]  # M(수컷)의 경우
        
        # 원본 데이터와 원-핫 인코딩 결합
        full_input = one_hot_encoded + input_data
        
        # SageMaker 런타임 클라이언트 생성
        runtime_client = boto3.client('sagemaker-runtime')
        
        # 입력 데이터를 CSV 형식의 문자열로 변환
        payload = ','.join(map(str, full_input))
            
        # 엔드포인트 호출
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=payload
        )
        
        # 응답 결과 처리
        result = response['Body'].read().decode('utf-8')
        predictions = json.loads(result)
        
        return predictions
        
    except Exception as e:
        print(f"추론 중 오류 발생: {e}")
        raise

# 성별을 지정할 수 있는 버전의 함수
def predict_abalone_with_sex(endpoint_name, input_data, sex='M'):
    """
    성별을 지정하여 전복 데이터 추론
    
    Parameters:
    endpoint_name (str): SageMaker 엔드포인트 이름
    input_data (list): [Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight]
    sex (str): 'M'(수컷), 'F'(암컷), 'I'(미성숙) 중 하나
    
    Returns:
    predictions: 추론 결과
    """
    try:
        # 성별에 따른 원-핫 인코딩
        sex_encoding = {
            'M': [1, 0, 0],  # 수컷
            'F': [0, 1, 0],  # 암컷
            'I': [0, 0, 1]   # 미성숙
        }
        
        if sex not in sex_encoding:
            raise ValueError("sex must be one of 'M', 'F', or 'I'")
            
        # 원본 데이터와 원-핫 인코딩 결합
        full_input = sex_encoding[sex] + input_data
        
        # SageMaker 런타임 클라이언트 생성
        runtime_client = boto3.client('sagemaker-runtime')
        
        # 입력 데이터를 CSV 형식의 문자열로 변환
        payload = ','.join(map(str, full_input))
            
        # 엔드포인트 호출
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=payload
        )
        
        # 응답 결과 처리
        result = response['Body'].read().decode('utf-8')
        predictions = json.loads(result)
        
        return predictions
        
    except Exception as e:
        print(f"추론 중 오류 발생: {e}")
        raise

# 사용 예시
try:
    # 테스트용 샘플 데이터
    sample_data = [0.455,0.365,0.095,0.514,0.2245,0.101,0.15]
    
    # 각 성별에 대해 추론 실행
    for sex in ['M', 'F', 'I']:
        predictions = predict_abalone_with_sex(endpoint_name, sample_data, sex)
        print(f"\n성별: {sex}")
        print(f"입력 데이터: {sample_data}")
        print(f"예측 결과: {predictions}")
    
    # 여러 데이터에 대한 배치 추론
    batch_data = [
        [0.455,0.365,0.095,0.514,0.2245,0.101,0.15],
        [0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07],
        [0.53,0.42,0.135,0.677,0.2565,0.1415,0.21]
    ]
    
    print("\n배치 추론 실행:")
    for data in batch_data:
        prediction = predict_abalone_with_sex(endpoint_name, data, 'M')  # 예시로 모두 수컷으로 가정
        print(f"입력: {data}")
        print(f"예측: {prediction}\n")
        
except Exception as e:
    print(f"오류 발생: {e}")
    raise

## 5. Delete Endpoint
- 추가적인 비용이 나가지 않게 엔드포인트 삭제 
    - 엔드포인트 컨피그, 모델도 같이 삭제 합니다

In [ ]:
import boto3

def delete_endpoint_and_resources(endpoint_name):
    try:
        sagemaker_client = boto3.client('sagemaker')
        
        # 1. 엔드포인트 설정 이름 조회
        endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        endpoint_config_name = endpoint_response['EndpointConfigName']
        
        # 2. 모델 이름 조회
        endpoint_config_response = sagemaker_client.describe_endpoint_config(
            EndpointConfigName=endpoint_config_name
        )
        model_name = endpoint_config_response['ProductionVariants'][0]['ModelName']
        
        # 3. 엔드포인트 삭제
        print(f"Deleting endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        
        # 4. 엔드포인트 설정 삭제
        print(f"Deleting endpoint configuration: {endpoint_config_name}")
        sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
        
        # 5. 모델 삭제
        print(f"Deleting model: {model_name}")
        sagemaker_client.delete_model(ModelName=model_name)
        
        print("Successfully deleted all resources")
        
    except sagemaker_client.exceptions.ClientError as e:
        print(f"Error occurred: {str(e)}")
        raise

# 사용 예시
delete_endpoint_and_resources(endpoint_name)